In [280]:
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))
from pathlib import Path

import pandas as pd
import numpy as np

import scipy
from scipy import optimize, integrate
from scipy import interpolate

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import container
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
from matplotlib.ticker import LinearLocator, FormatStrFormatter

import time
import math
import random
import multiprocessing as mp
import os
import subprocess
from shutil import copyfile
import copy
from cubature import cubature

%load_ext autoreload
%autoreload 2

from modules_protein_structure.nd_ps_2 import *
from modules_protein_structure.nd_ps_misc import *

from modules_ns_bim_spherical.geometry import *
from modules_ns_bim_spherical.hybrid_model_oc_v1 import *
from modules_ns_bim_spherical.numerical_Derjaguin_v1 import *
from modules_ns_bim_spherical.oc_plots_v1 import *
from modules_ns_bim_spherical.orthogonal_collocation_v1 import *
from modules_ns_bim_spherical.v2_ns_bim_helper_plus import *
from modules_ns_bim_spherical.monte_carlo_functions import *

from biopandas.pdb import PandasPdb
import varname

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. From buried_charges.ipynb (in extra_ubuntu_spa/correlation_aex/)

## Identification of surface atoms

In [281]:
prot_folders = os.listdir('ns_bim_spherical/structure_files_from_depth_work/pqr_structure_files/')
prot_folders.sort()
prot_folders

['adh_pH_7',
 'blg_pH_7',
 'cat_pH_7',
 'lys_pH_5',
 'lys_pH_7',
 'lys_pH_9',
 'ova_pH_7']

In [282]:
dfs_pdb = {}
dfs_pdb_dep = {} # Depths of each atom in the .pdb file beneath the VdW surface
dfs_pqr = {} # .pqr file (protein structure) -- note that in this format, BioPandas 
             # loads the partial charge into the occupancy column
dfs_pqr_dep = {} # Depths of each atom in the .pqr file beneath the VdW surface

for f in prot_folders:
    pdb_path = f'./ns_bim_spherical/structure_files_from_depth_work/pdb_structure_files/{f}/'
    pdb_file = pdb_path + [i for i in os.listdir(pdb_path) if '.pdb' in i][0]
    dep_file = pdb_path + [i for i in os.listdir(pdb_path) if '_atom.dep' in i][0]
    dfs_pdb[f]     = PandasPdb().read_pdb(pdb_file)
    dfs_pdb_dep[f] = pd.read_fwf(dep_file)
    
    pqr_path = f'./ns_bim_spherical/structure_files_from_depth_work/pqr_structure_files/{f}/'
    pqr_file = pqr_path + [i for i in os.listdir(pqr_path) if '.pqr' in i][0]
    dep_file = pqr_path + [i for i in os.listdir(pqr_path) if '_atom.dep' in i][0]
    dfs_pqr[f]     = PandasPdb().read_pdb(pqr_file)
    dfs_pqr_dep[f] = pd.read_fwf(dep_file)

In [283]:
for f in prot_folders:
    for i, icont in dfs_pdb[f].df['ATOM'].iterrows():
        df_dep = dfs_pdb_dep[f]
        depth = df_dep.loc[df_dep['atom#']==icont.atom_number, 'depth'].iloc[0]
        dfs_pdb[f].df['ATOM'].at[i, 'depth'] = depth

    for i, icont in dfs_pdb[f].df['HETATM'].iterrows():
        df_dep = dfs_pdb_dep[f]
        depth = df_dep.loc[df_dep['atom#']==icont.atom_number, 'depth'].iloc[0]
        dfs_pdb[f].df['HETATM'].at[i, 'depth'] = depth
        
    for i, icont in dfs_pqr[f].df['ATOM'].iterrows():
        df_dep = dfs_pqr_dep[f]
        depth = df_dep.loc[df_dep['atom#']==icont.atom_number, 'depth'].iloc[0]
        dfs_pqr[f].df['ATOM'].at[i, 'depth'] = depth

    for i, icont in dfs_pqr[f].df['HETATM'].iterrows():
        df_dep = dfs_pqr_dep[f]
        depth = df_dep.loc[df_dep['atom#']==icont.atom_number, 'depth'].iloc[0]
        dfs_pqr[f].df['HETATM'].at[i, 'depth'] = depth

### Manually add Zn and Fe groups

In [284]:
f = 'adh_pH_7'
df = dfs_pdb[f].df['HETATM'].copy()
df = df[df.atom_name=='ZN']
df.occupancy = 2.0
dfs_pqr[f].df['ATOM'] = pd.concat([dfs_pqr[f].df['ATOM'], df], ignore_index=True)

f = 'cat_pH_7'
df = dfs_pdb[f].df['HETATM'].copy()
df = df[df.atom_name=='FE']
df.occupancy = 2.0
dfs_pqr[f].df['ATOM'] = pd.concat([dfs_pqr[f].df['ATOM'], df], ignore_index=True)

### Center protein and find surface atoms

In [285]:
for f in prot_folders:
    df = dfs_pqr[f].df['ATOM']
    (x_cent, y_cent, z_cent) = (df.x_coord.mean(), df.y_coord.mean(), df.z_coord.mean())
    
    df['x_orig'] = df.x_coord.copy()
    df['y_orig'] = df.y_coord.copy()
    df['z_orig'] = df.z_coord.copy()
    
    df.x_coord -= x_cent
    df.y_coord -= y_cent
    df.z_coord -= z_cent

In [286]:
# Assign unique sequential residue numbers 
for f in prot_folders:
    df = dfs_pqr[f].df['ATOM']
    residue_number = -1    
    
    my_residue_number = 0
    for i, icont in df.iterrows():
        if icont.residue_number != residue_number:
            my_residue_number += 1
            residue_number = icont.residue_number
        df.at[i, 'my_residue_number'] = my_residue_number

In [287]:
surf_atom = {}

for f in prot_folders:
    df = dfs_pqr[f].df['ATOM']
    try:
        df.drop(columns=['blank_1', 'alt_loc', 'blank_2', 'chain_id', 'insertion', 
                    'blank_3', 'b_factor', 'blank_4', 'segment_id', 'element_symbol',
                    'charge'], inplace=True)
    except:
        pass
    df_surf = df[df.depth < 3.2].copy()
    df_surf.reset_index(drop=True, inplace=True)
    surf_atom[f] = df_surf

# 2. From spherical_NSBIM.ipynb

## Identification of most attractive orientations

In [288]:
base_names = ['adh_pH_7', 'blg_pH_7', 'cat_pH_7', 'lys_pH_5', 'lys_pH_7', 'lys_pH_9', 
              'ova_pH_7']

### For data based on PDB2PQR partial charges

In [289]:
results_dir = 'results_pqr_partial_charges'
z0          = 3.0e-10 

nsbim_data_dir   = 'ns_bim_spherical/' + results_dir + '/ns_bim_data/'
integrand_dir    = 'ns_bim_spherical/' + results_dir + '/integrand_results/'
integral_dir     = 'ns_bim_spherical/' + results_dir + '/integral_results/'
sum_contribs_dir = 'ns_bim_spherical/' + results_dir + '/summed_integral_contribs/'
images_dir       = 'ns_bim_spherical/' + results_dir + '/images/'

In [290]:
protein_folders = os.listdir(nsbim_data_dir)
protein_folders.sort()
# protein_folders

In [291]:
elements_dict, node_dict, charge_dict = get_elem_and_node_dicts(nsbim_data_dir, 
                                                                protein_folders)
elements_dict = fill_elem_dict(elements_dict, node_dict, protein_folders)

In [311]:
path = './ns_bim_spherical/results_pqr_partial_charges/integral_results/'
files = os.listdir(path)
files_200 = [i for i in files if '50' in i and (('phq' in i) or ('sep' in i))]
files_200.sort()

dict_vals = {}
dict_orient = {}

for f in files_200:
    name = f[4:f.find('.')]
    df_element = elements_dict[name]
    
    dict_vals[f] = pd.read_csv(path + f)
    dict_vals[f].rename(columns={'potential_V':'potential_V_2'}, inplace=True)
    
    df = pd.concat([dict_vals[f], df_element], axis=1, join='inner')
    df.sort_values(by='integral', inplace=True, ascending=False)
    df.reset_index(inplace=True, drop=True)
    
    dict_orient[name] = df[:200].copy() # PARAMETER

# 3. Residues of closest approach in most attractive orientations

In [294]:
def get_magnitude(x, y, z):
    return np.sqrt(x**2.0 + y**2.0 + z**2.0)

def get_line_distance(n_x, n_y, n_z, x, y, z):
    """https://mathworld.wolfram.com/Point-LineDistance3-Dimensional.html
       https://en.wikipedia.org/wiki/Cross_product"""
    
    v1_x = x
    v1_y = y
    v1_z = z
    
    v2_x = x - n_x
    v2_y = y - n_y
    v2_z = z - n_z
    
    v3_x = n_x
    v3_y = n_y
    v3_z = n_z
    
    v4_x = v1_y * v2_z - v1_z * v2_y
    v4_y = v1_z * v2_x - v1_x * v2_z
    v4_z = v1_x * v2_y - v1_y * v2_x
    
    num_mag = get_magnitude(v4_x, v4_y, v4_z)
    den_mag = get_magnitude(v3_x, v3_y, v3_z)
    distance = num_mag/den_mag
    return distance

In [354]:
selection_all = {}
selection_pruned = {}

for f in prot_folders:
    long_name = [i for i in files_200 if f in i][0]
    name      = long_name[4:long_name.find('.')]
    df_orient = dict_orient[name]
    
    rows_all = []
    rows_pruned = []
    
    for i, cont in df_orient.iterrows():
        df_surf = surf_atom[f].copy()
        df_surf['d_to_vec'] = get_line_distance(cont.n_x_nm, cont.n_y_nm, cont.n_z_nm, 
                                                df_surf.x_coord, df_surf.y_coord, df_surf.z_coord)
        if cont.n_x_nm == 0:
            df_surf['prod_x'] = 1.0
        else:
            df_surf['prod_x'] = cont.n_x_nm*df_surf.x_coord
        
        if cont.n_y_nm == 0:
            df_surf['prod_y'] = 1.0
        else:
            df_surf['prod_y'] = cont.n_y_nm*df_surf.y_coord
            
        if cont.n_z_nm == 0:
            df_surf['prod_z'] = 1.0
        else:
            df_surf['prod_z'] = cont.n_z_nm*df_surf.z_coord
        
        df_surf.sort_values(by='d_to_vec', inplace=True)
        df_sort = df_surf[(df_surf.prod_x > 0) & (df_surf.prod_y > 0) & (df_surf.prod_z > 0)] # make sure the vector is pointing the right way
        df_sort.reset_index(drop=True, inplace=True)
        
        for j in range(15): # PARAMETER - to (hopefully) find the charge in the patch - "pruned"
            extra = pd.Series({'orientation_num':i, 'integral':cont.integral,
                              'potential_V':cont.potential_V, 
                               'charge_C_m2':cont.charge_C_m2})            
            rows_pruned.append(df_sort.loc[j].append(extra)) 
        
        for j in range(1): # PARAMETER - to pick out just the closest residue - "all"
            extra = pd.Series({'orientation_num':i, 'integral':cont.integral,
                              'potential_V':cont.potential_V, 
                               'charge_C_m2':cont.charge_C_m2})            
            rows_all.append(df_sort.loc[j].append(extra)) 
    
    df = pd.DataFrame(rows_all)
    df.reset_index(inplace=True, drop=True)
    df.drop_duplicates(subset=['my_residue_number'], inplace=True, ignore_index=True)    
    df.drop(columns=['prod_x', 'prod_y', 'prod_z'], inplace=True)
    df.to_csv('./ns_bim_spherical/orientations_to_explore/all_residues/' + f + '.csv', index=False)
    selection_all[f] = df
    
    df = pd.DataFrame(rows_pruned)
    df.reset_index(inplace=True, drop=True)
    df.drop_duplicates(subset=['my_residue_number'], inplace=True, ignore_index=True)    
    df.drop(columns=['prod_x', 'prod_y', 'prod_z'], inplace=True)
    if 'lys' in f: # CEX - basic residues
        df_pruned = df[(df.residue_name == 'HIS') | (df.residue_name == 'LYS') | (df.residue_name == 'ARG')]
    else: # AEX - acidic residues
        df_pruned = df[(df.residue_name == 'ASP') | (df.residue_name == 'GLU') | (df.residue_name == 'CYS') | (df.residue_name == 'TYR')]
    df_pruned.to_csv('./ns_bim_spherical/orientations_to_explore/charged_residues/' + f + '.csv', index=False)
    selection_pruned[f] = df_pruned

In [355]:
for f in prot_folders:
    df_all = selection_all[f]
    df_pruned = selection_pruned[f]
    print(f, len(df_all), len(df_pruned))

adh_pH_7 64 23
blg_pH_7 14 10
cat_pH_7 42 20
lys_pH_5 8 4
lys_pH_7 5 3
lys_pH_9 7 3
ova_pH_7 10 9


In [356]:
selection_all[f]

,record_name,atom_number,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,occupancy,line_idx,depth,x_orig,y_orig,z_orig,my_residue_number,d_to_vec,orientation_num,integral,potential_V,charge_C_m2
0,ATOM,341,O,ALA,46,-3.085573,15.112957,-10.898786,-0.55,397,2.880,13.007,47.087,12.468,23.0,0.822387,0.0,114.594209,-0.120958,-0.283016
1,ATOM,357,HB2,ASN,47,-2.197573,17.574957,-10.658786,0.00,413,2.600,13.895,49.549,12.708,24.0,0.716737,4.0,111.455032,-0.100673,-0.096241
2,ATOM,373,HG3,GLU,48,-4.550573,14.750957,-8.576786,0.00,429,2.600,11.542,46.725,14.790,25.0,1.118506,12.0,107.772115,-0.136244,-0.224093
3,ATOM,380,O,ASN,49,-2.795573,10.776957,-4.931786,-0.55,436,2.880,13.297,42.751,18.435,26.0,1.336810,13.0,107.630971,-0.132028,-0.199924
4,ATOM,403,HG12,ILE,50,-2.049573,8.471957,-7.393786,0.00,459,3.182,14.043,40.446,15.973,27.0,0.694980,15.0,105.747533,-0.105381,-0.214848
5,ATOM,329,HB3,HIS,45,-4.017573,10.710957,-8.982786,0.00,385,2.839,12.075,42.685,14.384,22.0,0.959347,30.0,97.828830,-0.102664,-0.124505
6,ATOM,5926,HH12,ARG,387,-0.210573,11.938957,-9.810786,0.40,5767,3.182,15.882,43.913,13.556,362.0,0.692583,53.0,85.086244,-0.068141,0.070781
7,ATOM,4621,OH,TYR,297,-4.032573,6.892957,-4.913786,-0.49,4487,2.880,12.060,38.867,18.453,274.0,1.149858,95.0,68.749363,-0.148221,-0.301711
8,ATOM,5956,HG22,VAL,389,-0.406573,15.808957,-5.366786,0.00,5797,2.894,15.686,47.783,18.000,364.0,1.225352,126.0,60.005228,-0.072232,0.273444
9,ATOM,461,HG,CYS,53,0.105427,-2.616043,1.906214,0.29,517,2.600,16.198,29.358,25.273,30.0,0.735122,153.0,50.621884,-0.087406,0.140111


In [357]:
selection_pruned[f]

,record_name,atom_number,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,occupancy,line_idx,depth,x_orig,y_orig,z_orig,my_residue_number,d_to_vec,orientation_num,integral,potential_V,charge_C_m2
4,ATOM,373,HG3,GLU,48,-4.550573,14.750957,-8.576786,0.00,429,2.600,11.542,46.725,14.790,25.0,2.258941,0.0,114.594209,-0.120958,-0.283016
7,ATOM,448,HB3,TYR,52,-2.826573,2.722957,-3.660786,0.00,504,2.915,13.266,34.697,19.706,29.0,2.669177,0.0,114.594209,-0.120958,-0.283016
8,ATOM,4621,OH,TYR,297,-4.032573,6.892957,-4.913786,-0.49,4487,2.880,12.060,38.867,18.453,274.0,2.777046,0.0,114.594209,-0.120958,-0.283016
12,ATOM,3191,OE2,GLU,214,-0.572573,27.617957,-8.742786,-0.55,3102,2.880,15.520,59.592,14.624,191.0,3.207037,100.0,66.713813,-0.063871,0.315560
15,ATOM,4586,HB3,GLU,295,-5.957573,7.336957,-2.780786,0.00,4452,2.839,10.135,39.311,20.586,272.0,3.367510,119.0,62.857275,-0.165398,-0.400970
16,ATOM,3958,OE2,GLU,261,5.141427,17.211957,-15.350786,-0.55,3854,2.880,21.234,49.186,8.016,238.0,4.659577,133.0,56.318094,-0.085311,-0.120393
19,ATOM,4008,OE1,GLU,264,4.690427,10.616957,-11.367786,-0.55,3904,2.880,20.783,42.591,11.999,241.0,4.726911,147.0,53.609651,-0.066196,0.062971
20,ATOM,461,HG,CYS,53,0.105427,-2.616043,1.906214,0.29,517,2.600,16.198,29.358,25.273,30.0,0.735122,153.0,50.621884,-0.087406,0.140111
26,ATOM,1289,HA,ASP,98,3.372427,-15.768043,6.871214,0.00,1238,2.600,19.465,16.206,30.238,75.0,3.363278,153.0,50.621884,-0.087406,0.140111
